In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import RandomSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda:0"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)

load nci


In [5]:
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = RandomSampler(res, train_index, test_index, null_mask)
    model = nihgcn(
        adj_mat=sampler.train_data,
        cell_exprs=exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    ).to(args.device)
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)], ignore_index=True)
    predict_datas = pd.concat(
        [predict_datas, translate_result(predict_data)], ignore_index=True
    )

epoch:   0 loss:0.701695 auc:0.5168
epoch:  20 loss:0.469723 auc:0.7707
epoch:  40 loss:0.379306 auc:0.8040
epoch:  60 loss:0.332603 auc:0.8227
epoch:  80 loss:0.307724 auc:0.8287
epoch: 100 loss:0.292880 auc:0.8345
epoch: 120 loss:0.282697 auc:0.8382
epoch: 140 loss:0.277515 auc:0.8392
epoch: 160 loss:0.275161 auc:0.8382
epoch: 180 loss:0.268885 auc:0.8418
epoch: 200 loss:0.264243 auc:0.8427
epoch: 220 loss:0.259674 auc:0.8449
epoch: 240 loss:0.257785 auc:0.8443
epoch: 260 loss:0.254698 auc:0.8451
epoch: 280 loss:0.252487 auc:0.8450
epoch: 300 loss:0.252069 auc:0.8437
epoch: 320 loss:0.251767 auc:0.8430
epoch: 340 loss:0.246796 auc:0.8456
epoch: 360 loss:0.247195 auc:0.8449
epoch: 380 loss:0.245877 auc:0.8455
epoch: 400 loss:0.243011 auc:0.8456
epoch: 420 loss:0.242558 auc:0.8438
epoch: 440 loss:0.241168 auc:0.8432
epoch: 460 loss:0.238731 auc:0.8435
epoch: 480 loss:0.237602 auc:0.8443
epoch: 500 loss:0.236399 auc:0.8422
epoch: 520 loss:0.233984 auc:0.8435
epoch: 540 loss:0.233493 auc

In [6]:
true_datas.to_csv("true_nci.csv")
predict_datas.to_csv("pred_nci.csv")